In [ ]:
# import csv
# from io import StringIO
# import pyperclip


# data = [
#     ["Alice", 24, "Engineer"],
#     ["Bob", 30, "Data Scientist"],
#     ["Charlie", 22, "Designer"]
# ]
# headers = ["Name", "Age", "Job"]

# # Create a CSV-formatted string
# output = StringIO()
# writer = csv.writer(output)
# writer.writerow(headers)  # Write headers
# writer.writerows(data)  # Write rows

# csv_output = output.getvalue()
# print(csv_output)
# pyperclip.copy(csv_output)  # Now paste into Excel!

In [ ]:
import pandas as pd
import numpy as np
import os
from glob import glob
import matplotlib.pyplot as plt

import csv
from io import StringIO
import pyperclip

from tabulate import tabulate
printtab = lambda x : print(tabulate(x, headers='firstrow'))


In [ ]:
"""
    Reference for remembering which index corresponds to which demographic

    # CVD
    self.output_cvd[cy,0] = self.num_cvd_cases["total"]
    self.output_cvd[cy,1] = self.num_cvd_cases["black"]
    self.output_cvd[cy,2] = self.num_cvd_cases["nonblack"]
    self.output_cvd[cy,3] = self.num_cvd_cases["pov"]
    self.output_cvd[cy,4] = self.num_cvd_cases["nonpov"]

    # LC
    self.output_lc[cy,0] = self.num_lc_cases["total"]
    self.output_lc[cy,1] = self.num_lc_cases["black"]
    self.output_lc[cy,2] = self.num_lc_cases["nonblack"]
    self.output_lc[cy,3] = self.num_lc_cases["pov"]
    self.output_lc[cy,4] = self.num_lc_cases["nonpov"]
"""


In [ ]:
base_dir = "/Users/nick/Documents/Gillings_work/uncertainty_analysis_data/uncertainty_analysis_2025-01-09_07-22-27-435007"
base_dir = "/Users/nick/Documents/Gillings_work/uncertainty_analysis_data/uncertainty_analysis_2025-01-30_08-15-25-366670"
base_dir = "/Users/nick/Documents/Gillings_work/uncertainty_analysis_data/uncertainty_analysis_2025-02-04_21-26-48-562904"
base_dir = "/Users/nick/Documents/Gillings_work/uncertainty_analysis_data/uncertainty_analysis_2025-02-17_18-48-05-068917"
base_dir = "/Users/nick/Documents/Gillings_work/uncertainty_analysis_data/uncertainty_analysis_2025-02-22_16-30-32-358613"
base_dir = "/Users/nick/Documents/Gillings_work/uncertainty_analysis_data/uncertainty_analysis_2025-07-15_15-18-06-138213"
base_dir = "/Users/nick/Documents/Gillings_work/uncertainty_analysis_data/uncertainty_analysis_2025-07-30_09-50-22-950580"
output_dir = os.path.join(base_dir, "disease_modeling_outputs")
outputs_dirs = [os.path.join(output_dir, f"option_{i}") for i in range(6)]

In [ ]:
opt0_dir = outputs_dirs[0]
opt1_dir = outputs_dirs[1]

In [ ]:
start_year = 2016
years = [2016,2024,2025,2026,2027,2030,2035,2055]
year_inds = list(np.array(years) - start_year)

In [ ]:
print(opt0_dir)

In [ ]:
# Grab the disease incidence data
all_data_all_options_LC = []
for opt in (0,1,2,3,4):
    opt_dir = outputs_dirs[opt]
    all_data = []
    for f in glob(opt_dir + "/*lc.npy"):
        arr = np.load(f) # shape is (years, demographic groups)
        all_data.append(arr)
    all_data_all_options_LC.append(all_data)

all_data_all_options_LC = np.array(all_data_all_options_LC)

all_data_all_options_CVD = []
for opt in (0,1,2,3,4):
    opt_dir = outputs_dirs[opt]
    all_data = []
    for f in glob(opt_dir + "/*cvd.npy"):
        arr = np.load(f) # shape is (years, demographic groups)
        all_data.append(arr)
    all_data_all_options_CVD.append(all_data)

all_data_all_options_CVD = np.array(all_data_all_options_CVD)

all_data_all_options_TOTAL = []
for opt in (0,1,2,3,4):
    opt_dir = outputs_dirs[opt]
    all_data = []
    for f in glob(opt_dir + "/*total.npy"):
        arr = np.load(f) # shape is (years, demographic groups)
        all_data.append(arr)
    all_data_all_options_TOTAL.append(all_data)

all_data_all_options_TOTAL = np.array(all_data_all_options_TOTAL)

# options, runs, years, demographic groups
print(all_data_all_options_LC.shape)


In [ ]:
# Define the demographic groups
demo_names = [
    "all demographics",
    "black",
    "nonblack",
    "pov",
    "nonpov",
]

option_names = [
    "SQ",
    "B1",
    "B2",
    "B3",
    "B4",
]

In [ ]:
# disease modeling happens before transitions happens before aging
# This is the year where we want to extract our measurement:
year_of_interest = 65 # 2035 (youngest people in cohort are 18 this year) - 2016 (start year) + (64-18) (how many years they need to reach 64 years old)

In [ ]:
def get_mean_lower_upper(arr, axis, decimals):
    """
    Given an array, return the mean across an axis, as well as the lower and upper bounds of a 95% confidence interval
    Round outputs to the nearest tenth
    """
    return (
        np.around(np.mean(arr, axis=axis),decimals=decimals),
        np.around(np.percentile(arr, 2.5, axis=axis),decimals=decimals),
        np.around(np.percentile(arr, 97.5, axis=axis),decimals=decimals),
    )


def incidence_table(adao_disease, adao_totals):
    counts = np.copy(adao_disease)
    totals = np.copy(adao_totals)

    counts = counts[:,:, year_of_interest,:] 
    totals = totals[:,:, year_of_interest,:]

    # now the indices are options, runs, demographics

    # get proportions and change from status quo
    proportions = counts / totals
    proportions *= 100 # for percentages

    all_sq = np.tile(counts[0,:,:],(counts.shape[0],1,1))
    assert (all_sq.shape == counts.shape)
    change = (counts - all_sq) / all_sq
    change *= 100 # for percentages

    # expected = change < 0
    # unexpected = change > 1
    # print(np.count_nonzero(unexpected))
    # print(np.count_nonzero(expected))


    # get mean, upper, lower
    counts_mean, counts_lower, counts_upper = get_mean_lower_upper(counts, axis=1, decimals=-1)
    proportions_mean, proportions_lower, proportions_upper = get_mean_lower_upper(proportions, axis=1, decimals=3)
    change_mean, change_lower, change_upper = get_mean_lower_upper(change, axis=1, decimals=3)

    # now the rows are options, demos

    # print table
    # the rows are demographic groups, columns are ban options.
    # need count, percentage, and %change (for ban options)

    header = ["Demographic"]
    for i, option_name in enumerate(option_names):
        header += [f"{option_name} Count", f"{option_name} Percentage"]
        if (i != 0):
            header += [f"{option_name} % Change from SQ"]

    rows= []
    for i, demo_name in enumerate(demo_names):
        row = [f"{demo_name}"]
        for j, _ in enumerate(option_names):
            row += [f"{counts_mean[j,i]} ({counts_lower[j,i]}, {counts_upper[j,i]})"]
            row += [f"{proportions_mean[j,i]}% ({proportions_lower[j,i]}, {proportions_upper[j,i]})"]
            if (j != 0):
                # row += [f"{change_mean[j,i]}% ({change_lower[j,i]}, {change_upper[j,i]})"]

                change_mean = (counts_mean[j,i] - counts_mean[0,i]) / counts_mean[0,i]
                change_mean = np.around(change_mean * 100, decimals=3)
                row += [f"{change_mean}%"]

        rows += [row]

    table = [header] + rows
    print(tabulate(table, headers='firstrow', tablefmt='grid'))

    output = StringIO()
    writer = csv.writer(output)
    writer.writerow(header)  # Write headers
    writer.writerows(rows)  # Write rows
    csv_output = output.getvalue()
    print(csv_output)
    pyperclip.copy(csv_output)  # Now paste into Excel!

In [ ]:
print("CVD: ")
incidence_table(all_data_all_options_CVD, all_data_all_options_TOTAL)

In [ ]:
print("LC: ")
incidence_table(all_data_all_options_LC, all_data_all_options_TOTAL)

In [ ]:
assert False

In [ ]:
data1=means[0]
data2=means[1]

# Create x-axis values
x = np.arange(len(data1))

# Create the plot
plt.figure(figsize=(10, 6))

# Plot both lines with different colors and labels
plt.plot(x, data1, color='blue', linestyle='-', 
         linewidth=2, markersize=8, label='Status Quo')
plt.plot(x, data2, color='red', linestyle='-', 
         linewidth=2, markersize=8, label='Ban Option 1')

# Customize the plot
plt.xlabel('Year', fontsize=12)
plt.ylabel('Incidence of LC', fontsize=12)

# Add grid and legend
plt.grid(True, linestyle='--', alpha=0.7)
plt.legend()

# Show the plot
plt.show()

In [ ]:

all_data_all_options = []
for opt in (0,1):
    opt_dir = outputs_dirs[opt]
    all_data = []
    for f in glob(opt_dir + "/*cvd.npy"):
        arr = np.load(f) # shape is (years, demographic groups)
        arr = arr[:,0] # all the years, whole population    
        ##print(arr)
        all_data.append(arr)
    all_data_all_options.append(all_data)

all_data_all_options = np.array(all_data_all_options)
print(all_data_all_options.shape)

In [ ]:

means = []
for opt in range(len(all_data_all_options)):
    means.append(np.mean(all_data_all_options[opt,:,:], axis=0)) # mean over all the different runs
means = np.array(means)
print(means.shape)

In [ ]:

data1=means[0]
data2=means[1]

# Create x-axis values
x = np.arange(len(data1))

# Create the plot
plt.figure(figsize=(10, 6))

# Plot both lines with different colors and labels
plt.plot(x, data1, color='blue', linestyle='-', 
         linewidth=2, markersize=8, label='Status Quo')
plt.plot(x, data2, color='red', linestyle='-', 
         linewidth=2, markersize=8, label='Ban Option 1')

# Customize the plot
plt.xlabel('Year', fontsize=12)
plt.ylabel('Incidence of LC', fontsize=12)

# Add grid and legend
plt.grid(True, linestyle='--', alpha=0.7)
plt.legend()

# Show the plot
plt.show()

In [ ]:

opt0_f = (sorted(glob(opt0_dir + "/*ls.npy")))[0]
opt1_f = (sorted(glob(opt1_dir + "/*ls.npy")))[0]

print(opt0_f)
print(opt1_f)

data1 = np.load(opt0_f)[:,0]
data2 = np.load(opt1_f)[:,0]
# Create x-axis values
x = np.arange(len(data1))

# Create the plot
plt.figure(figsize=(10, 6))

# Plot both lines with different colors and labels
plt.plot(x, data1, color='blue', linestyle='-', 
         linewidth=2, markersize=8, label='Status Quo')
plt.plot(x, data2, color='red', linestyle='-', 
         linewidth=2, markersize=8, label='Ban Option 1')

# Customize the plot
plt.xlabel('Year', fontsize=12)
plt.ylabel('Incidence of LC', fontsize=12)

# Add grid and legend
plt.grid(True, linestyle='--', alpha=0.7)
plt.legend()

# Show the plot
plt.show()

# for i,f in enumerate(sorted(glob(dir + "/*cvd.npy"))):
#     arr = np.load(f)
#     print(arr.shape)
#     data = np.diff(arr[:,0])

#     # Create x-axis values (indices of the data points)
#     x = np.arange(len(data))

#     # Create the plot
#     plt.figure(figsize=(10, 6))
#     plt.plot(x, data, marker='o', linestyle='-', linewidth=2, markersize=8)

#     # Customize the plot
#     plt.title('Data Visualization', fontsize=14)
#     plt.xlabel('Index', fontsize=12)
#     plt.ylabel('Value', fontsize=12)

#     # Add grid
#     plt.grid(True, linestyle='--', alpha=0.7)

#     # Show the plot
#     plt.show()
    